In [ ]:
from pathlib import Path
import geopandas as gpd
import shutil
from RA2CE_Run_functions_module import load_region_shapefile, load_networks, filter_network_by_region, create_segments_simple, move_output_file, copy_and_prepare_flood_map
from ra2ce.network.network_config_data.network_config_data import NetworkConfigData, NetworkSection, HazardSection
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.analysis.damages.damages import AnalysisSectionDamages
from ra2ce.analysis.analysis_config_data.enums.analysis_damages_enum import AnalysisDamagesEnum
from ra2ce.analysis.analysis_config_data.enums.event_type_enum import EventTypeEnum
from ra2ce.analysis.analysis_config_data.enums.damage_curve_enum import DamageCurveEnum
from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisConfigData
from ra2ce.ra2ce_handler import Ra2ceHandler


c:\SnapVolumesTemp\MountPoints\{45c63495-0000-0000-0000-100000000000}\{dd207255-f76a-4e73-8bed-3e64afdf945f}\SVROOT\envs\ra2ce_env_brs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
region_list = ["Vallei en Veluwe"]

In [3]:

region_shapefile = Path(r"P:\bovenregionale-stresstest-hwn\Data\Hazard_maps\Gebiedsindeling obv waterschappen EXPORT 4dec24.gpkg")
HWN_network = Path(r"P:\bovenregionale-stresstest-hwn\Data\Shapes netwerkschakels\Shapes netwerkschakels\HWN_netwerkindeling.shp")
NWB_network = Path(r"P:\bovenregionale-stresstest-hwn\Data\Road_data\Rijkswegen_uit_nwb\rijkswegen.shp")

In [ ]:
for region in region_list:
    print(f"Processing region: {region} hazard map")
    flood_map_folder = Path(rf"P:\bovenregionale-stresstest-hwn\Data\Hazard_maps\Hazard_maps-in_use\{region}\Basisscenario\waterdiepte")
    flood_map_files = list(flood_map_folder.glob("*max_wd_merge_clipNL.tif")) + list(flood_map_folder.glob("*max_wd_merge_clipNL.tiff"))
    flood_duration_folder = Path(rf"P:\bovenregionale-stresstest-hwn\Data\Hazard_maps\Hazard_maps-in_use\{region}\Basisscenario\Duur")
    Flood_duration_files = list(flood_duration_folder.glob("*duur_merge_clipNL.tif")) + list(flood_map_folder.glob("*duur_merge_clipNL.tiff"))
    
    if flood_map_files:
        flood_map_path = flood_map_files[0]
        print(f"Using flood map: {flood_map_path}")
    else:
        print("No TIFF files found in the folder.")
        
    if Flood_duration_files:
        flood_duration_path = Flood_duration_files[0]
        print(f"Using flood duration map: {flood_duration_path}")
    else:
        flood_duration_path = None
        print("No TIFF files found in the folder.")

    print("Processing networks - clipping to region and joining HWN and NWB networks")
    region_gdf, filtered_region = load_region_shapefile(region_shapefile, region)
    joined_network = load_networks(HWN_network, NWB_network)
    
    print(joined_network.columns)
    code = 'NWSCODE_HWN'  # This is what we use to clip ex: Networkschals inbetween regions should not be clipped
    filtered_network_gdf = filter_network_by_region(joined_network, filtered_region,code)
    
    print("Segmenting networks into 100m segments")
    print("Here we chose the NWB ID as the metric for segmenting the network into 100m segments.")
    segmentation_metric = 'id_NWB' #determine which metric to use for 100m segments
    segmented_gdfs = []
    grouped_networks = filtered_network_gdf.groupby(segmentation_metric)

    id_counter = 1
    for nwscode, group in grouped_networks:
        group_segments = []
        for idx, row in group.iterrows():
            segments_gdf = create_segments_simple(row.geometry, segment_length=100)
            for seg_idx, segment_row in segments_gdf.iterrows():
                new_row = row.copy()
                new_row.geometry = segment_row.geometry
                new_row['segment_id'] = f"{idx}_{seg_idx}"
                new_row['original_id'] = idx
                new_row['segment_number'] = seg_idx
                new_row['segment_length'] = segment_row.geometry.length
                new_row['highway'] = "motorway"
                new_row['lanes'] = 1
                new_row['REF_ID'] = id_counter 
                id_counter += 1  
                
                group_segments.append(new_row)
        if group_segments:
            group_gdf = gpd.GeoDataFrame(group_segments, crs=group.crs)
            segmented_gdfs.append(group_gdf)

    if segmented_gdfs:
        segmented_network_gdf = gpd.pd.concat(segmented_gdfs, ignore_index=True)
    else:
        segmented_network_gdf = filtered_network_gdf.copy()

    print("creating inputs for RA2CE run")
    region_dir = Path(rf"P:\bovenregionale-stresstest-hwn\Analysis\{region}")
    input_dir = region_dir.joinpath("Inputs")
    output_path = region_dir.joinpath("Outputs")
    static_path = input_dir.joinpath("static")
    overlay_path = static_path.joinpath("output_graph")
    network_path = static_path.joinpath("network")
    hazard_path = static_path.joinpath("hazard")
    
    

    for path in [region_dir, input_dir, static_path, network_path, hazard_path, output_path, output_path]:
        path.mkdir(parents=True, exist_ok=True)

    segmented_network_file = network_path.joinpath(f"{region}_segmented_network.shp")
    columns_to_drop = [col for col in segmented_network_gdf.columns if col.startswith('geometry_') and col != 'geometry']
    segmented_network_clean = segmented_network_gdf.drop(columns=columns_to_drop)
    segmented_network_clean.to_file(segmented_network_file)

    print("Preprocessing Flood Map")
    filtered_region_area = region_gdf[region_gdf['name'] == region]
    copy_and_prepare_flood_map(flood_map_path, hazard_path, filtered_region_area, nodata_value=-9999)
    if flood_duration_path:
        copy_and_prepare_flood_map(flood_duration_path, hazard_path, filtered_region_area, nodata_value=-9999)

    
    all_maps = list(hazard_path.glob("*.tif"))
    print(all_maps)
    
    # So the flood depth map would always be processed first
    hazard_map = []

    # Add the 'merge' map first (required)
    merge_maps = [Path(file) for file in all_maps if "max_wd_merge" in file.name.lower()]
    if not merge_maps:
        raise FileNotFoundError("No hazard map with 'merge' in the filename was found.")
    hazard_map.append(merge_maps[0])  # Assuming only one merge map

    # Add the 'duur' map if it exists (optional)
    duur_maps = [Path(file) for file in all_maps if "duur" in file.name.lower()]
    if duur_maps:
        hazard_map.append(duur_maps[0]) 

    root_dir = input_dir
    network_section = NetworkSection(
        source=SourceEnum.SHAPEFILE,
        primary_file=[network_path.joinpath(f"{region}_segmented_network.shp")],
        file_id="REF_ID",
        link_type_column="highway",
        save_gpkg=True
    )

    hazard = HazardSection(
        hazard_map=hazard_map,
        aggregate_wl=AggregateWlEnum.MAX,
        hazard_crs="EPSG:28992"
    )

    network_config_data = NetworkConfigData(
        root_path=root_dir,
        static_path=static_path,
        output_path=output_path,
        network=network_section,
        hazard=hazard
    )
    
    section_damage = [AnalysisSectionDamages(
    name='HZ_damage',
    analysis=AnalysisDamagesEnum.DAMAGES,
    event_type=EventTypeEnum.EVENT,
    damage_curve=DamageCurveEnum.HZ,
    save_gpkg=True,
    save_csv=True,
    )]
    
    analysis_config_data = AnalysisConfigData(analyses=section_damage, root_path=root_dir, output_path=output_path)
    analysis_config_data.input_path = root_dir.joinpath("input_data")

    print("Creating RA2CE handler and running analysis")
    handler = Ra2ceHandler.from_config(network_config_data, analysis_config_data)
    handler.configure()
    handler.run_analysis()

    move_output_file(overlay_path, output_path, "base_network_hazard.gpkg")
    


Processing region: Vallei en Veluwe hazard map
Using flood map: P:\bovenregionale-stresstest-hwn\Data\Hazard_maps\Hazard_maps-in_use\Vallei en Veluwe\Basisscenario\waterdiepte\samengevoegd_max_inundatiediepte_5m_max_wd_merge_clipNL.tif
Using flood duration map: P:\bovenregionale-stresstest-hwn\Data\Hazard_maps\Hazard_maps-in_use\Vallei en Veluwe\Basisscenario\Duur\samengevoegd_hersteltijd_5m_uren_duur_merge_clipNL.tif
Processing networks - clipping to region and joining HWN and NWB networks
HWN network columns:
Index(['NWBVERSIE', 'NWBWVKID', 'NETWERKCAT', 'NWSCODE', 'NWSNAAM', 'STDLNR',
       'STDLNAAM', 'STDLPOS', 'VKWNAAM', 'VKWCAT', 'VKWFUNCTIE', 'OBJNAAM',
       'NWBWGNR', 'NWBWGDL', 'NWBHECTO', 'NWBRPE', 'NWBBST', 'NWBBKM',
       'NWBEKM', 'NWBGME', 'NWBDISTR', 'NWBDIENST', 'NWBLENGTE', 'NWBWGAW',
       'NWBWGHM', 'UPDATE', 'OBJECTID', 'LAST_UPDAT', 'SE_ANNO_CA',
       'KNOOPPUNTT', 'NETWERKSCH', 'NETWERKS00', 'NETWERKS01', 'NETWERKS02',
       'ST_LENGTH(', 'geometry'],
   

C:\Users\gunaratn\AppData\Local\Temp\ipykernel_14192\2388508769.py:79: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  segmented_network_clean.to_file(segmented_network_file)


Preprocessing Flood Map
[WindowsPath('P:/bovenregionale-stresstest-hwn/Analysis/Vallei en Veluwe/Inputs/static/hazard/samengevoegd_hersteltijd_5m_uren_duur_merge_clipNL.tif'), WindowsPath('P:/bovenregionale-stresstest-hwn/Analysis/Vallei en Veluwe/Inputs/static/hazard/samengevoegd_max_inundatiediepte_5m_max_wd_merge_clipNL.tif')]
Creating RA2CE handler and running analysis


100%|██████████| 2314/2314 [00:00<00:00, 257170.63it/s]
2025-08-18 05:21:36 PM - [avg_speed_calculator.py:176] - root - WARNING - No valid file found with average speeds P:\bovenregionale-stresstest-hwn\Analysis\Vallei en Veluwe\Inputs\static\output_graph\avg_speed.csv, calculating and saving them instead.
2025-08-18 05:21:36 PM - [avg_speed_calculator.py:176] - root - WARNING - No valid file found with average speeds P:\bovenregionale-stresstest-hwn\Analysis\Vallei en Veluwe\Inputs\static\output_graph\avg_speed.csv, calculating and saving them instead.
Graph hazard overlay with samengevoegd_max_inundatiediepte_5m_max_wd_merge_clipNL: 100%|██████████| 1159/1159 [00:19<00:00, 59.01it/s]
Graph fraction with hazard overlay with samengevoegd_max_inundatiediepte_5m_max_wd_merge_clipNL: 100%|██████████| 1159/1159 [08:59<00:00,  2.15it/s] 
Graph hazard overlay with samengevoegd_hersteltijd_5m_uren_duur_merge_clipNL: 100%|██████████| 1159/1159 [00:17<00:00, 66.12it/s]
Graph fraction with hazar

Successfully moved base_network_hazard.gpkg from P:\bovenregionale-stresstest-hwn\Analysis\Vallei en Veluwe\Inputs\static\output_graph to P:\bovenregionale-stresstest-hwn\Analysis\Vallei en Veluwe\Outputs
